In [150]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.transforms import *
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import numpy as np

## Load data (Q7)
Use the dataloaders to load both the train and the test set into large tensors:
one for the instances, one for the labels. Split the training data into 50 000 training instances
and 10 000 validation instances. Then write a training loop that loops over batches of 16
instances at a time.

In [151]:
TRAIN_SIZE = 50000
VALIDATION_SIZE = 10000
ROOT = '../data' # .. to not have data folder within notebooks folder
BATCH_SIZE = 64 # TODO same size for training + validation set?
EPOCHS = 3

LEARNING_RATE = 0.001

### Data transformer

See on PyTorch discussion forum [here](https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/2) on choice for 0.137 (mean) and 0.3081 (std.) for normalization. Official PyTorch example code for MNIST also uses these parameters. 

In [203]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

In [204]:
# For question 9: augmentations.
#                                       transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
#                                       transforms.ColorJitter(brightness=0.2, contrast=0.2),
transform_train = transforms.Compose([transforms.RandomRotation(15),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,))])

### Training / validation data

(Note: Using ```.dataset``` on **train_set** or **validation_set** will return the original 60000 instances, instead use the specified dataloaders further below)

In [205]:
from copy import copy

dataset = torchvision.datasets.MNIST(root = ROOT,
                                 train = True,
                                 download = True)
train_set, validation_set = torch.utils.data.random_split(dataset, [TRAIN_SIZE, VALIDATION_SIZE])

train_set.dataset = copy(dataset)

train_set.dataset.transform = transform_train
validation_set.dataset.transform = transform


train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           num_workers = 2)

validation_loader = torch.utils.data.DataLoader(validation_set,
                                                batch_size = BATCH_SIZE,
                                                shuffle = True,
                                                num_workers = 2)

In [206]:
train_set.dataset.transform

Compose(
    RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
    ToTensor()
    Normalize(mean=(0.1307,), std=(0.3081,))
)

In [207]:
validation_set.dataset.transform

Compose(
    ToTensor()
    Normalize(mean=(0.1307,), std=(0.3081,))
)

### Test data

In [208]:
test_set = dataset = torchvision.datasets.MNIST(root = ROOT,
                                 train = False,
                                 transform = transform,
                                 download = True)

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size = BATCH_SIZE,
                                          shuffle = True,
                                          num_workers = 2)

## Network (Q8)

In [209]:
log_interval = 100

In [210]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = 1) # size: batch, 1, 28, 28
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1) # (batch, 16, 14, 14)
        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1) # (batch, 32, 7, 7)
        self.fc1 = nn.Linear(64*3*3, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # (batch, 16, 28, 28) # TODO klopt het dat relu en maxpool zijn omgedraaid?
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # (batch, 32, 14, 14)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2) # (batch, 64, 7, 7)
        x = torch.flatten(x, 1) # (batch, 64, 3, 3)
        x = self.fc1(x)
        return F.log_softmax(x, dim = 1)

In [211]:
network = Net()
optimizer = optim.Adam(network.parameters(), lr=LEARNING_RATE)

In [212]:
train_losses = []
train_counter = []
val_losses = []
val_counter = [i*TRAIN_SIZE for i in range(EPOCHS + 1)]
val_acc = []

In [213]:
# TODO Make sure to have "results" directory in "CNN/"
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), '../results/model_Q8.pth')
            torch.save(optimizer.state_dict(), '../results/optimizer_Q8.pth')

In [214]:
def test():
    network.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            output = network(data)
            val_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        val_loss /= VALIDATION_SIZE
        val_losses.append(val_loss)
        val_acc.append(float(100. * correct / VALIDATION_SIZE))
        print('\nValidation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            val_loss, correct, VALIDATION_SIZE,
            100. * correct / VALIDATION_SIZE))

In [215]:
test()
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test()


Validation set: Avg. loss: 2.3227, Accuracy: 644/10000 (6%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.303464
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.343425
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.107883
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.069025
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.164441
Train Epoch: 1 [32000/50000 (64%)]	Loss: 0.047794
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.114620
Train Epoch: 1 [44800/50000 (90%)]	Loss: 0.034199

Validation set: Avg. loss: 0.0668, Accuracy: 9802/10000 (98%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.057902
Train Epoch: 2 [6400/50000 (13%)]	Loss: 0.369460
Train Epoch: 2 [12800/50000 (26%)]	Loss: 0.078846
Train Epoch: 2 [19200/50000 (38%)]	Loss: 0.050628
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.060554
Train Epoch: 2 [32000/50000 (64%)]	Loss: 0.036584
Train Epoch: 2 [38400/50000 (77%)]	Loss: 0.088717
Train Epoch: 2 [44800/50000 (90%)]	Loss: 0.111465

Validation set: Avg. loss: 0.0412, Accuracy: 9875/10000 (99%)

Train Epoch: 3 [0/5000

# Plotting

In [216]:
# Save progress for plotting.
np.savetxt("../results/augmented_training_stats.csv", [p for p in zip(train_counter, train_losses)], delimiter=',', fmt='%s')
np.savetxt("../results/augmented_validation_stats.csv", [p for p in zip(val_counter, val_losses, val_acc)], delimiter=',', fmt='%s')

In [78]:
import subprocess

res = subprocess.call("Rscript ../scripts/plot_training.R", shell=True)
res


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Rows: 24 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (2): X1, X2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 4 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (3): X1, X2, X3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning messages:
1: Removed 24 rows containing missing values (geom_point). 
2: Removed 24 rows containing missing values (geom_text_repel). 
Warning messages:
1: Removed 24 rows containing missing values (geom_point). 
2: R

0